In [2]:
import pandas as pd
import geopandas as gp
from src.utils.map_vis import geojson_per_row
import folium
from shapely.ops import cascaded_union
from shapely.geometry import Polygon, MultiPolygon

%matplotlib inline

# Mexico

In [1]:
from src.utils import fix_spanish_encoding

## urban areas

In [3]:
# http://www.inegi.org.mx/geo/contenidos/geoestadistica/m_g_0.aspx
shp_urban = gp.read_file('data/mexico/mglu2014v6_2/mglu2015v6_2.shp')
shp_state = gp.read_file('data/mexico/mge2014v6_2/mge2015v6_2.shp')
shp_munic = gp.read_file('data/mexico/mgm2014v6_2/mgm2015v6_2.shp')

# get Urban id
shp_urban.NOM_LOC = shp_urban.NOM_LOC.apply(fix_spanish_encoding)
shp_urban['URBAN_ID'] = shp_urban.CVE_ENT + shp_urban.CVE_MUN + shp_urban.CVE_LOC

# get the full data frame
shp_all = shp_urban.merge(shp_state.drop('geometry', axis=1)).merge(shp_munic.drop('geometry', axis=1))

Fix the holes in urban areas of DF mexico city, 

In [5]:
polys = cascaded_union(shp_all[shp_all.CVE_ENT=='09'].geometry)
p0=polys[0]
p0 = Polygon(p0.exterior, [p0.interiors[115]])
p4 = Polygon(polys[4].exterior)

mex_city_geo = MultiPolygon([p0, p4]+list(polys[1:4]) + list(polys[5:]))
mexico_city = {'CVE_ENT': '09', 'CVE_MUN': '000', 'CVE_LOC': '0000', 
               'NOM_LOC': 'Mexico city in DC', 'NOM_ENT': 'Distrito Federal',
               'NOM_MUN': 'Mexico city in DC', 'concat': '09000', 'URBAN_ID' :'090000000',
               'geometry': mex_city_geo}
mexico_city = gp.GeoDataFrame([mexico_city])

### municipality head

In [25]:
cities = ['Acapulco, Guerrero', 'Cancún, Quintana Roo', 'Ciudad Juárez, Chihuahua', 'Chihuahua, Chihuahua', 
          'Cuernavaca, Morelos', 'Culiacán, Sinaloa', 'Guadalajara, Jalisco', 'Mexicali, Baja California', 
          'Monterrey, Nuevo León', 'Morelia, Michoacán', 'Nuevo Laredo, Tamaulipas', 'Oaxaca, Oaxaca', 
          'Tijuana, Baja California', 'Toluca, Estado de México', 'Villahermosa, Tabasco', 'Valle de Mexico, Distrito Federal']

idx = [742, 3396, 454, 431, 2095, 3515, 1186, 48, 2280, 1943, 3815, 2373, 71, 1771, 3709]
selected_munic_head = shp_all.loc[idx].append(mexico_city, ignore_index=True, sort=False)

cols = ['URBAN_ID', 'NOM_ENT', 'NOM_MUN','NOM_LOC',  'CVE_ENT', 'CVE_MUN', 'CVE_LOC', 'geometry']
selected_munic_head = selected_munic_head[cols]
selected_munic_head['name'] = cities

In [50]:
selected_munic_head.to_file('data/mex_16_munic_head.geojson', driver="GeoJSON")

### municipality urban areas

In [47]:
selected_urban_unmerged = shp_all[shp_all['concat'].isin(shp_all.loc[idx]['concat'])]

In [48]:
selected_urbans = []

for i in idx:
    concat_value = shp_all.loc[i,'concat']
    munic = shp_all[shp_all['concat']==concat_value]
    geometry = cascaded_union(munic.geometry)
    selected_urbans.append([munic.CVE_ENT.iloc[0], munic.CVE_MUN.iloc[0], munic.NOM_ENT.iloc[0], munic.NOM_MUN.iloc[0], geometry])

columns=['CVE_ENT','CVE_MUN', 'NOM_ENT','NOM_MUN','geometry']
selected_urbans = gp.GeoDataFrame(selected_urbans, columns=columns).append(mexico_city[columns])
selected_urbans['name'] = cities

In [49]:
selected_urbans.to_file('data/mex_16_munic_urban_merge.geojson', driver="GeoJSON")

## metro

In [52]:
metro = gp.read_file('data/mexico/Mapa_de_delimitacion_de_zonas_metropolitanas_2010/ZM_2010.shp')
sun_ids = [31, 21, 32, 28, 24, 11, 12, 41, 37, 3, 17,2,25,45]
sun_names = ['Monterrey, Nuevo León', 'Guadalajara, Jalisco', 'Oaxaca de Juárez, Oaxaca', 'Cuernavaca, Morelos', 'Toluca de Lerdo, México', 'Juárez, Chihuahua', 'Chihuahua, Chihuahua', 'Villahermosa, Tabasco', 'Cancún, Quintana Roo', 'Mexicali, Baja California', 'Acapulco de Juárez, Guerrero', 'Tijuana, Baja California', 'Morelia, Michoacán de Ocampo', 'Nuevo Laredo, Tamaulipas','Culiacán Rosales, Sinaloa', 'Valle de Mexico, Distrito Federal']

suns = []
for cve,name in zip(sun_ids,sun_names):
    sun = metro[metro.CVE_SUN==cve]
    geometry = cascaded_union(sun.geometry.tolist())
    pob = sun.POB_2010.sum()
    suns.append({'cname': name, 'cve_sun': cve, 'population': pob, 'nom_sun':sun.NOM_SUN.unique()[0], 'geometry':geometry})

sun = metro[(metro.CVE_SUN==13) & (metro.CVE_ENT=='09')]
geometry = cascaded_union(sun.geometry.tolist())
pob = sun.POB_2010.sum()
suns.append({'cname': sun_names[-1], 'cve_sun': 13, 'population': pob, 'nom_sun':sun.NOM_SUN.unique()[0], 'geometry':geometry})

In [53]:
# use municipal area for metro-culiacan instead of munic-head. Other metro are municipal level
# culiacan_sinaloa = selected_localidades_mex[selected_localidades_mex.URBAN_ID=='250060001'].to_crs(metro.crs).iloc[0].to_dict()
# population from: https://www.citypopulation.de/php/mexico-metro.php
culiacan_sinaloa = shp_munic[shp_munic['concat']=='25006'].to_crs(metro.crs).iloc[0].to_dict()
suns.append({'cname': sun_names[-2], 'cve_sun': 99, 'population': 858638, 'nom_sun':sun_names[-2], 
             'geometry':culiacan_sinaloa['geometry']})

In [54]:
suns=gp.GeoDataFrame(suns)
suns.crs = metro.crs
suns = suns.to_crs(epsg=4326)

In [55]:
suns = suns[['cname','population', 'cve_sun', 'geometry']].sort_values('population', ascending=False)

In [56]:
suns.to_file('data/mex_16_metropolitans.geojson', driver="GeoJSON")

## maps

In [61]:
some_map = folium.Map(location=[24.062199, -102.870324], zoom_start=5)
geojson_per_row(suns.reset_index(), name='metro', tip_cols=['cname','population','cve_sun'], some_map=some_map)
geojson_per_row(selected_urban_unmerged, name='munic-urban-unmerged', color='grey', tip_cols=['URBAN_ID', 'NOM_ENT', 'NOM_MUN','NOM_LOC'], some_map=some_map)
geojson_per_row(selected_urbans, name='munic-urban', color='yellow', tip_cols=['NOM_ENT', 'NOM_MUN','name'], some_map=some_map)
geojson_per_row(selected_munic_head, name='munic-head', color='red', tip_cols=['URBAN_ID', 'NOM_ENT', 'NOM_MUN','NOM_LOC','name'], some_map=some_map)

folium.LayerControl().add_to(some_map)
some_map.save('maps/metro_urban_areas.html')

## mexico country

In [ ]:
%%time
mex = cascaded_union(shp_state.geometry.values)

In [ ]:
new_polys = [Polygon(p.exterior) for p in mex]
new_polys = MultiPolygon(new_polys)
mex_gpdf = gp.GeoDataFrame([[mex,]], columns=['geometry'])
clean_mex = gp.GeoDataFrame([[new_polys,]], columns=['geometry'])

In [ ]:
maps = folium.Map(location=[19.381495, -99.139095], zoom_start=6)
folium.GeoJson(mex_gpdf.to_json(), name='raw cascad union').add_to(maps)
folium.GeoJson(clean_mex.to_json(), name='keep exterior only').add_to(maps)
folium.LayerControl().add_to(maps)
maps.save('maps/mexico_country.html')

In [ ]:
clean_mex.to_file('data/mex_country.geojson', driver='GeoJSON')